In [1]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import engine
import pickle
import numpy as np

In [2]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_features(trackID):
        return sp.audio_features(trackID)[0]

def get_popularity(trackID):
    return sp.track(trackID)['popularity']

# def get_markets(trackID):
#     foo = sp.track(trackID)['available_markets']
#     if foo != None:
#         return 1
#     else:
#         return 0

In [3]:
def get_markets(trackID):
    foo = sp.track(trackID)['available_markets']
    if foo != []:
        return 1
    else:
        return 0

In [16]:
con = sqlite3.connect("song_list_v3_raw.db")

In [17]:
# Load the data into a DataFrame
X = pd.read_sql_query("SELECT * from songs", con)
con.close()

In [5]:
X = pickle.load( open( "./final_data/DF_v4.pkl", "rb"))

In [6]:
X.head()

,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,songid,speechiness,tempo,time_signature,track,valence,genres,genres_stripped,tokens
0,0.001300,Karkkiautomaatti,0.487,157307.0,0.678,0.055100,9.0,0.0846,-7.780,1.0,5PS5dpaLogPzYU9hWiWyZb,0.0495,149.940,4.0,Tanssi vaan,0.870,"['finnish indie', 'suomi rock']","'finnish indie', 'suomi rock'","[['finnish, indie',, 'suomi, rock']]"
1,0.000045,Hudson Mohawke,0.662,138960.0,0.823,0.952000,4.0,0.3430,-1.711,0.0,41RpZW2lxAdnqDd2nMBzLQ,0.0662,177.745,4.0,No One Could Ever,0.621,"['bass music', 'scottish electronic', 'scottis...","'bass music', 'scottish electronic', 'scottish...","[['bass, music',, 'scottish, electronic',, 'sc..."
2,0.002760,Kris Kross,0.859,221200.0,0.741,0.000000,11.0,0.3250,-12.329,0.0,3GsS8jzoixpCnp4jDWCEvb,0.2710,98.082,4.0,2 Da Beat Ch'yall,0.529,"['hip hop', 'new jack swing', 'rap']","'hip hop', 'new jack swing', 'rap'","[['hip, hop',, 'new, jack, swing',, 'rap']]"
3,0.348000,Jorge Negrete,0.361,173573.0,0.483,0.000003,7.0,0.1770,-6.875,1.0,0kq75szR7uDEYrZkT2c4Ry,0.0287,94.538,4.0,El hijo del pueblo,0.682,"['bolero', 'mariachi', 'ranchera']","'bolero', 'mariachi', 'ranchera'","[['bolero',, 'mariachi',, 'ranchera']]"
4,0.340000,Tiger Lou,0.533,318467.0,0.302,0.179000,10.0,0.1110,-10.308,1.0,2HyFpkX9J7vv3OZNDaraHZ,0.0307,134.959,3.0,Pilots,0.294,['swedish indie rock'],'swedish indie rock',"[['swedish, indie, rock']]"


In [18]:
X = X.drop('level_0', axis=1)

In [19]:
X.head(1)

,index,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,songid,speechiness,tempo,time_signature,track,valence,genres,popularity,availability
0,0,0.0013,Karkkiautomaatti,0.487,157307.0,0.678,0.0551,9.0,0.0846,-7.78,1.0,5PS5dpaLogPzYU9hWiWyZb,0.0495,149.94,4.0,Tanssi vaan,0.87,"['finnish indie', 'suomi rock']",0,1


In [20]:
engine = create_engine('sqlite:///song_list_v3_22.db', echo=False)
X.to_sql('songs', con=engine)
engine.execute('SELECT * FROM songs;').fetchone()

(0, 0, 0.0013, 'Karkkiautomaatti', 0.487, 157307.0, 0.678, 0.0551, 9.0, 0.0846, -7.78, 1.0, '5PS5dpaLogPzYU9hWiWyZb', 0.0495, 149.94, 4.0, 'Tanssi vaan', 0.87, "['finnish indie', 'suomi rock']", '0', '1')

In [8]:
markets_list = []
for row_index in X['songid'][0:1000]:
    songid = row_index
#     #print(songid)
#     popularity = get_popularity(songid)
#     #print(popularity)
    markets = get_markets(songid)
    markets_list.append(markets)
#     #print(markets)
#     try:
#         engine.execute(f'UPDATE songs SET popularity = "{popularity}", availability = "{markets}" WHERE songid = \"{songid}\"')
#         authorize()
#     except:
#         print(f"couldn't insert {markets} or {popularity} into table")
#         continue

retrying ...1secs
retrying ...2secs
retrying ...3secs


In [9]:
len(markets_list)

1000

In [10]:
markets_list.count(1) 

984

In [12]:
1000 / 984

1.016260162601626